In [2]:
import re

In [3]:
import numpy as np
import pandas as pd

In [4]:
from Bio import SeqIO
import os

In [5]:
from tqdm import tqdm

In [6]:
import json

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
import matplotlib.pyplot as plt

In [9]:
from sklearn.metrics import roc_curve, auc 

In [10]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier  
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression

In [11]:
import random
from phylodm import PhyloDM
import dendropy

In [12]:
import seaborn as sns
from scipy import stats

In [14]:
from scipy.stats import entropy

### import embedding

In [13]:
co_embedding = pd.read_csv("../../data/social_niche_embedding_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
co_embedding = co_embedding.drop("<unk>")

embed_cos = cosine_similarity(co_embedding)
embed_cos = pd.DataFrame(data=embed_cos, index=co_embedding.index, columns=co_embedding.index)

phy_embedding = pd.read_csv("../../data/Embedding_list/PCA_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
phy_embedding = phy_embedding.loc[co_embedding.index]

embed_cos_phy = cosine_similarity(phy_embedding)
embed_cos_phy = pd.DataFrame(data=embed_cos_phy, index=phy_embedding.index, columns=phy_embedding.index)

In [18]:
tree = dendropy.Tree.get_from_path('../SNE_overview/Data//tree.tre', schema='newick')
pdm = PhyloDM.load_from_dendropy(tree)
dm = pdm.dm(norm=False)
labels = pdm.taxa()
dm = pd.DataFrame(data=dm, index=labels, columns=labels) 
dm = dm.loc[co_embedding.index.values, co_embedding.index.values]

### SNE with phylogeny

In [15]:
taxonomy = pd.read_csv("../Pretraining_data_profile/Data/taxmap_slv_ssu_ref_nr_138.2.txt", sep="\t", low_memory=False)

In [16]:
acc = []
for i in range(taxonomy.shape[0]):
    temp = taxonomy.iloc[i]
    acc.append(f"{temp[0]}.{temp[1]}.{temp[2]}")

taxonomy = taxonomy.loc[:, "path"].str.split(';', expand=True)
taxonomy.index = acc
taxonomy = taxonomy.iloc[:, 0: 7]
taxonomy = taxonomy.loc[co_embedding.index.values, ]
taxonomy.columns = ["k", "p", "c", "o", "f", "g", "s"]
taxonomy = taxonomy.loc[taxonomy.k == "Bacteria"]
taxonomy = taxonomy.loc[:, ["p", "c", "o", "f", "g"]]
remove_tax = ["Incertae Sedis", "Unknown Family", ]

In [19]:
names = ["Phylum", "Class", "Order", "Family", "Genus"]
phy_dist_mean = []
embed_cosine_mean = []
phy_dist_sd = []
embed_cosine_sd = []
keep_tax = []
tax = []
for j in range(0, 5):
    group = taxonomy.iloc[:, j].unique()
    group = group[[i not in remove_tax for i in group]]
    for i in group:
        
        fid_pick = taxonomy.loc[taxonomy.iloc[:, j] == i].index.values
        
        if len(fid_pick) > 1:
            
            keep_tax.append(i)
            tax.append(names[j])
            
            temp = embed_cos.loc[fid_pick, fid_pick]
            upper_triangle = np.triu(temp, k=1)
            upper_triangle = pd.DataFrame(data=upper_triangle)
            upper_triangle['id_1'] = upper_triangle.index.tolist()
            upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
            upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
            upper_triangle.columns = ["id_1", "id_2", "value"]
            embed_cosine_mean.append(np.mean(upper_triangle["value"].values))
            embed_cosine_sd.append(np.std(upper_triangle["value"].values))
            
            temp = dm.loc[fid_pick, fid_pick]
            upper_triangle = np.triu(temp, k=1)
            upper_triangle = pd.DataFrame(data=upper_triangle)
            upper_triangle['id_1'] = upper_triangle.index.tolist()
            upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
            upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
            upper_triangle.columns = ["id_1", "id_2", "value"]
            phy_dist_mean.append(np.mean(upper_triangle["value"].values))
            phy_dist_sd.append(np.std(upper_triangle["value"].values))
        

In [20]:
table_group = pd.DataFrame({"name": keep_tax, "phy_dist_mean":phy_dist_mean, "phy_dist_sd":phy_dist_sd, 
                            "embed_cosine_mean":embed_cosine_mean, "embed_cosine_sd":embed_cosine_sd,
                            "tax":tax})
table_group.to_csv("Data/tax_group.csv", index=None)

### vsearch

In [261]:
from Bio import SeqIO

def get_sequence_lengths(fasta_file):
    seq_lengths = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_lengths[record.id] = len(record.seq)  # 存储序列ID和长度
    return seq_lengths

# 示例调用
fasta_path = "../Traits_function/Data/feces_seq_16S_new.fasta"
length_dict = get_sequence_lengths(fasta_path)

In [251]:
colnames = ["query_id", "refer_id", "identity", "alignment_length", "mismatches", "gap_openings", "q.start",
            "q.end", "s.start", "s.end", "e-value", "bit_score"]

In [281]:
vsearch_out = pd.read_csv("Data/vsearch_blast.out", sep="\t", header=None)
vsearch_out.columns = colnames
vsearch_out.loc[:, "length"] = [length_dict[i] for i in vsearch_out.query_id.values]
vsearch_out.loc[:, "coverage"] = vsearch_out.alignment_length.values / vsearch_out.length.values

In [282]:
vsearch_out = vsearch_out.loc[vsearch_out["q.start"] == 1]
vsearch_out = vsearch_out.loc[vsearch_out["coverage"] >= 1]
vsearch_out.loc[:, "refer"] = [i.split("::")[1].split(":")[0] for i in vsearch_out.refer_id.values]

In [287]:
vsearch_out = vsearch_out.loc[vsearch_out.groupby('refer')['identity'].idxmax()]

In [292]:
vsearch_out.head()

,query_id,refer_id,identity,alignment_length,mismatches,gap_openings,q.start,q.end,s.start,s.end,e-value,bit_score,length,coverage,refer
1028,CP007065.253054.254560,16S_rRNA::AABF01000026.1:18737-20251(+),99.7,1507,4,0,1,1507,1,1514,-1,0,1507,1.0,AABF01000026.1
26,AJ239289.1.1355,16S_rRNA::ADBE01000137.1:300-1836(+),100.0,1355,0,0,1,1355,1,1536,-1,0,1355,1.0,ADBE01000137.1
684,AP018533.3254502.3256012,16S_rRNA::AP018533.1:3254488-3256017(-),100.0,1511,0,0,1,1511,1,1529,-1,0,1511,1.0,AP018533.1
1384,HQ767419.1.1452,16S_rRNA::AP025573.1:47380-48910(+),99.8,1452,3,0,1,1452,1,1530,-1,0,1452,1.0,AP025573.1
1622,JF160900.1.1352,16S_rRNA::AP025583.1:128195-129724(+),99.6,1352,6,0,1,1352,1,1529,-1,0,1352,1.0,AP025583.1


In [297]:
def select_sequences_by_ids(input_fasta, output_fasta, selected_ids):
    """
    根据序列名称选择指定的序列并保存到新文件
    
    参数:
        input_fasta (str): 输入 FASTA 文件路径
        output_fasta (str): 输出 FASTA 文件路径
        selected_ids (list): 需要选择的序列名称列表
    """
    
    # 读取输入文件并筛选序列
    selected_sequences = []
    for record in SeqIO.parse(input_fasta, "fasta"):
        if record.id in selected_ids:  # 如果序列名称在指定列表中
            record.id = record.id.split("::")[1].split(":")[0]
            selected_sequences.append(record)
    
    # 将筛选后的序列写入新文件
    SeqIO.write(selected_sequences, output_fasta, "fasta")
    print(f"已保存 {len(selected_sequences)} 条序列到 {output_fasta}")

# 示例调用
target_id = vsearch_out.refer_id.values
input_fasta = "Data/barrnap.fna"  # barrnapt 16S RNA of genome
output_fasta = "Data/pick_otu.fasta"  # 输出 FASTA 文件路径
select_sequences_by_ids(input_fasta, output_fasta, target_id) 

已保存 1011 条序列到 /home/dongbiao/word_embedding_microbiome/HGT/16S/pick_otu.fasta


In [28]:
def get_fasta_sequence_names(fasta_file):
    """
    提取单个 FASTA 文件中的所有序列名称
    
    参数:
        fasta_file (str): FASTA 文件路径
    
    返回:
        list: 序列名称列
    """
    sequence_names = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_names.append(record.id)
    return sequence_names

def process_fasta_folder(folder_path):
    """
    处理文件夹中的所有 FASTA 文件，生成字典
    
    参数:
        folder_path (str): 文件夹路径
    
    返回:
        dict: key 是基因组名称（文件名），value 是序列名称列表
    """
    fasta_dict = {}
    
    # 遍历文件夹中的所有文件
    for filename in tqdm(os.listdir(folder_path), desc="Processing"):
        if filename.endswith(".fna"):  # 检查文件扩展名
            file_path = os.path.join(folder_path, filename)
            genome_name = os.path.splitext(filename)[0]  # 去掉扩展名作为基因组名称
            sequence_names = get_fasta_sequence_names(file_path)
            fasta_dict[genome_name] = sequence_names
    
    return fasta_dict

In [298]:
# 读取 JSON 文件
with open('Data/fasta_dict.json', 'r', encoding='utf-8') as f:
    fasta_dict = json.load(f)  # 返回字典或列表

In [1]:
target_genome = {}
for target in tqdm(vsearch_out.refer.values, desc="Processing"):
    for key, value in fasta_dict.items():
        if target in value:
            target_genome[target] = key
            continue

In [301]:
vsearch_out.loc[:, "genome_id"] = [target_genome[i] for i in vsearch_out.refer.values]

In [309]:
vsearch_out = vsearch_out.loc[vsearch_out.groupby('genome_id')['identity'].idxmax()]

In [310]:
vsearch_out.head()

,query_id,refer_id,identity,alignment_length,mismatches,gap_openings,q.start,q.end,s.start,s.end,e-value,bit_score,length,coverage,refer,genome_id
26,AJ239289.1.1355,16S_rRNA::ADBE01000137.1:300-1836(+),100.0,1355,0,0,1,1355,1,1536,-1,0,1355,1.0,ADBE01000137.1,GCA_000176735.1_genomic
1028,CP007065.253054.254560,16S_rRNA::AABF01000026.1:18737-20251(+),99.7,1507,4,0,1,1507,1,1514,-1,0,1507,1.0,AABF01000026.1,GCA_000182945.1_genomic
504,JX424767.1.1450,16S_rRNA::CAPI01000044.1:3321-4819(-),99.9,1450,1,0,1,1450,1,1498,-1,0,1450,1.0,CAPI01000044.1,GCA_000333815.1_genomic
2086,ATWA01000156.1.1244,16S_rRNA::ATWA01000156.1:0-1245(-),100.0,1244,0,0,1,1244,1,1245,-1,0,1244,1.0,ATWA01000156.1,GCA_000421005.1_genomic
887,HE681338.1.1507,16S_rRNA::AZKM01000010.1:338-1852(+),99.5,1507,7,0,1,1507,1,1514,-1,0,1507,1.0,AZKM01000010.1,GCA_000510425.1_genomic


In [365]:
vsearch_out["query_id"] = [id_mapping[i] for i in vsearch_out.query_id.values]

In [366]:
vsearch_out.to_csv("Data/vsearch_res.csv", index=None)

In [313]:
genome_fid = {}
for i in range(vsearch_out.shape[0]):
    temp = vsearch_out.iloc[i].values
    genome_fid[temp[1]] = temp[0]

In [315]:
# 保存为 JSON 文件
with open("Data/genome_fid.json", "w") as f:
    json.dump(genome_fid, f, indent=4)  # indent 用于美化输出

In [368]:
genome_fid = {}
for i in range(vsearch_out.shape[0]):
    temp = vsearch_out.iloc[i].values
    genome_fid[temp[15]] = temp[0]

In [339]:
id_genome = {}
for i in range(vsearch_out.shape[0]):
    temp = vsearch_out.iloc[i].values
    id_genome[temp[14]] = temp[15]

### 16S identity

In [320]:
with open("Data/identity_matrix.txt", "r") as file:
    data = file.readlines()
value = []
genome_id = []
for line in data:
    genome_id.append(line.split(" ")[0])
    temp = line.split("\n")[0].split(" ")[1:]
    temp = [float(item) for item in temp if item != ""]
    value.append(temp)

In [324]:
identity_table = pd.DataFrame(data=value[1:], index=genome_id[1:], columns=genome_id[1:])
identity_table = identity_table.loc[vsearch_out.refer.values, vsearch_out.refer.values]

In [326]:
upper_triangle = np.triu(identity_table, k=1)
upper_triangle = pd.DataFrame(data=upper_triangle, index=identity_table.index, columns=identity_table.index)
upper_triangle['id_1'] = upper_triangle.index.tolist()
upper_triangle = pd.melt(upper_triangle, id_vars="id_1")
upper_triangle = upper_triangle.loc[upper_triangle.value != 0]
upper_triangle.columns = ["id_1", "id_2", "identity"]

In [327]:
upper_triangle = upper_triangle.loc[upper_triangle.identity < 97]

In [328]:
upper_triangle.head()

,id_1,id_2,identity
1001,ADBE01000137.1,AABF01000026.1,74.042273
2002,ADBE01000137.1,CAPI01000044.1,71.762347
2003,AABF01000026.1,CAPI01000044.1,72.029370
3003,ADBE01000137.1,ATWA01000156.1,79.598391
3004,AABF01000026.1,ATWA01000156.1,78.955823


In [340]:
genome_id_1 = []
genome_id_2 = []
for i in range(upper_triangle.shape[0]):
    temp = upper_triangle.iloc[i].values
    genome_id_1.append(id_genome[temp[0]])
    genome_id_2.append(id_genome[temp[1]])

In [341]:
genome_pairs = pd.DataFrame({"id_1":genome_id_1, "id_2":genome_id_2})

In [343]:
genome_pairs.head()

,id_1,id_2
0,GCA_000176735.1_genomic,GCA_000182945.1_genomic
1,GCA_000176735.1_genomic,GCA_000333815.1_genomic
2,GCA_000182945.1_genomic,GCA_000333815.1_genomic
3,GCA_000176735.1_genomic,GCA_000421005.1_genomic
4,GCA_000182945.1_genomic,GCA_000421005.1_genomic


In [344]:
genome_id = np.unique(genome_pairs.id_1.tolist() + genome_pairs.id_2.tolist())
genome_id = pd.DataFrame({"id": genome_id})
genome_id.to_csv("Data/genome_id.txt", index=None, header=None)

In [345]:
genome_pairs.to_csv("/home/dongbiao/word_embedding_microbiome/HGT/HGT_network/genome_pairs_vsearch.txt", sep=" ", index=None, header=None)

### HGT

In [ ]:
path="/home/dongbiao/word_embedding_microbiome/HGT/HGT_network/results/filtered"

hgt = []
genome_1 = genome_pairs.iloc[:,0].values
genome_2 = genome_pairs.iloc[:,1].values
for i in tqdm(range(genome_pairs.shape[0]), desc="Processing"):
    temp = genome_pairs.iloc[i].values
    try:
        table = pd.read_csv(f"{path}/{temp[0]}.fna_vs_{temp[1]}.fna_filtered.tsv")
        hgt.append(table.shape[0])
    except Exception:
        hgt.append(0)

identity = []
cosine_co = []
phy_dis = []
for i in range(len(id_1)):
    identity.append(identity_table.loc[id_1[i], id_2[i]])
    cosine_co.append(embed_cos.loc[id_1[i], id_2[i]])
    phy_dis.append(dm.loc[id_1[i], id_2[i]])

hgt_res = pd.DataFrame({"id_1":id_1, "id_2":id_2, "identity":identity, 
                        "cosine_co":cosine_co, "hgt":hgt, "phy_dis":phy_dis})
hgt_res.to_csv("Data/hgt.csv", index=None)

In [420]:
hgt_embed = pd.read_csv("Data/hgt.csv")

In [421]:
hgt_embed.head()

,id_1,id_2,identity,cosine_co,hgt,phy_dis
0,AJ239289.1.1355,CP007065.253054.254560,74.042273,0.386691,0,1.07411
1,AJ239289.1.1355,JX424767.1.1450,71.762347,-0.224302,0,1.06876
2,CP007065.253054.254560,JX424767.1.1450,72.029370,0.002536,0,1.04679
3,AJ239289.1.1355,ATWA01000156.1.1244,79.598391,-0.007625,0,1.04354
4,CP007065.253054.254560,ATWA01000156.1.1244,78.955823,0.303940,0,1.02157
